In [9]:
import numpy as np
import pandas as pd
import requests
from bs4 import BeautifulSoup
from urllib.request import urlretrieve
import re
from tqdm import tqdm
import os

In [10]:
from pdfminer.pdfinterp import PDFResourceManager, PDFPageInterpreter
from pdfminer.converter import TextConverter
from pdfminer.layout import LAParams
from pdfminer.pdfpage import PDFPage
from io import StringIO

def convert_pdf_to_txt(path):
    rsrcmgr = PDFResourceManager()
    retstr = StringIO()
    codec = 'utf-8'
    laparams = LAParams()
    device = TextConverter(rsrcmgr, retstr, laparams=laparams)
    fp = open(path, 'rb')
    interpreter = PDFPageInterpreter(rsrcmgr, device)
    password = ""
    maxpages = 0
    caching = True
    pagenos=set()

    for page in PDFPage.get_pages(fp, pagenos, maxpages=maxpages, password=password,caching=caching, check_extractable=True):
        interpreter.process_page(page)

    text = retstr.getvalue()

    fp.close()
    device.close()
    retstr.close()
    return text

In [ ]:
URL = 'http://www.stsci.edu/jwst/observing-programs/approved-gto-programs'
page = requests.get(URL)

soup = BeautifulSoup(page.content, 'html.parser')

all_targets = []

gto_pages = []
for link in soup.find_all('a'):
    if link.has_attr('href'):
        str_begin = '/jwst/observing-programs/program-information?id='
        if link.attrs['href'][:48] == str_begin:
            gto_page = 'http://www.stsci.edu/' + link.attrs['href'] # give better name
            gto_pages.append(gto_page)
            
for gto_page in tqdm(gto_pages, position=0, leave=True):
    ID = gto_page[-4:]
    pdf_link = f'http://www.stsci.edu/jwst/phase2-public/{ID}.pdf'
    link = 'http://www.stsci.edu/jwst/phase2-public/1243.pdf'
    urlretrieve(link, "tmp.pdf")
    text = convert_pdf_to_txt("tmp.pdf")
    start = text.find("Science Target") + len("Science Target")
    end =text.find("ABSTRACT")
    target_table = text[start:end]
    targets_numbered = re.sub("\n", "", target_table)
    target_concat = re.sub("[\(\[].*?[\)\]]", "", targets_numbered)
    targets = target_concat.split(' ')
    targets = [t for t in targets if len(t) > 0]
    all_targets.append(targets)
    os.remove('tmp.pdf')

 86%|████████▌ | 103/120 [04:14<00:40,  2.36s/it]